In [1]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from case_edits.epcase import EneryPlusCaseEditor
from case_edits.recipes.from_min import create_case

from case_edits.methods.materials import Materials

from case_edits.methods.subsurface import Subsurface, SubsurfaceAttributes, SubsurfaceType
from outputs.variables import OutputVars as ov

from geomeppy.patches import EpBunch

In [2]:
TEST_CASE = "tests/test11_afn"
SURFACE_CONSTRUCTIONS_CASE = "cases/constructions/ASHRAE_2005_HOF_Materials.idf"

In [3]:
outputs = [ov.surf_incident_solar_rad, ov.surf_net_thermal_rad, ov.zone_mean_air_temp, ov.in_surf_temp]
output_names = [o.value for o in outputs]
epcase = create_case(TEST_CASE, outputs=output_names)

cases/tests/test11_afn/out.idf
cases/tests/test11_afn/out.idf
Added 5 walls 
Added 4 walls 


In [4]:
mat = Materials(epcase)
mat.study_reference_idf(SURFACE_CONSTRUCTIONS_CASE)
mat.get_construction_materials('Light Furnishings')
c = mat.get_case_constructions()
# const = mat.case_constructions[-1]

In [5]:
west_surf = epcase.geometry.zones.B_West.walls.B_West_West_W03
int_surf = epcase.geometry.zones.B_West.walls.B_West_North_W02_1
north_surf = epcase.geometry.zones.B_North.walls.B_North_North_W02

window_const = epcase.idf.getobject("CONSTRUCTION", "Project External Window")
door_const = epcase.idf.getobject("CONSTRUCTION", 'Light Furnishings')



In [6]:
# windows
data = SubsurfaceAttributes(type=SubsurfaceType.WINDOW, length=1, height=1, construction=window_const, surface=west_surf) # type: ignore
Subsurface(epcase=epcase,attrs=data).create_surface()

data.surface = north_surf
Subsurface(epcase=epcase,attrs=data).create_surface()

In [7]:
# doors 
data = SubsurfaceAttributes(type=SubsurfaceType.DOOR, length=1, height=2, construction=door_const, surface=int_surf) # type: ignore
Subsurface(epcase=epcase,attrs=data).create_surface()


In [8]:
epcase.run_idf()


/Applications/EnergyPlus-22-2-0/energyplus --weather /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/weather/CA_PALO-ALTO-AP_724937S_20.epw --output-directory /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/tests/test11_afn/results --idd /Applications/EnergyPlus-22-2-0/Energy+.idd /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/base/01example/Minimal_45a2c6.idf

EnergyPlus Starting
EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.06.17 16:42
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 12/21 for DENVER_STAPLETON ANN HTG 99.6% CONDNS DB
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
Warming up {6}
Starting Simulation at 07/21 for DENVER_S

EnergyPlus Completed Successfully.
